# Distill Data ⚗️

Distill data from an open source model!

This is a basic synthetic data generator that uses SearXNG grounding, that does *not* require a model with tool calling capabilities. Feel free to adapt to your use case. 🙂

## Imports
These are the imports necessary to run the notebook, please also make sure you have Ollama along with the model you intend to use installed. If you don't have Ollama you can get it [here](https://ollama.com/) for Linux, MacOS, and Windows.

In [1]:
from tqdm import tqdm
import json

import ollama
from transformers import AutoTokenizer

/Users/dzurec/ai-projects/synthetic-data-factory/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Define Model and Inference Time Parameters
Ensure you type the *exact* name of your Ollama model as it appears with `ollama list`. 

In [2]:
model = "qwen3:1.7b-fp16"
# model = "gemma3:270m-it-qat"

# Uncomment for different parameters as needed
parameters = {
    # Keeps the first N tokens of the prompt
    # "num_keep": 5,
    # For reproducibility, should probably be left commented out unless for experimental purposes
    # "seed": 42,
    # Number of tokens the model is allowed to generate
    # "num_predict": 100,
    # Narrow sampling to the top K tokens
    # "top_k": 20,
    # Add up probabilites for tokens after softmax until top_p percentage is reached; discard the rest
    # "top_p": 0.9,
    # Sampling method following the formula threshold = min_p * [LIKLIHOOD OF MOST PROBABLE TOKEN]
    # Any token falling below the threshold is discarded
    # "min_p": 0.0,
    # Chooses "human-like responses" via entropy calculations; a value of 1.0 disables this sampling parameter
    # "typical_p": 0.7,
    # The window size for how far back in the context the model looks for the repeat penalty (the window)
    # "repeat_last_n": 33,
    # Sampling parameter that divides logits by the temperature, higher values result in more "creativity"; lower values result in less creativity
    # and favor the most statistically likely output.
    "temperature": 0.8,
    # Modifies logit scores based on if the logit would be repeated in the window defined by `repeat_last_n`
    # 1.0 disables this sampling parameter, less than 1.0 rewards repetiton, while greater than 1.0 penalizes repetition.
    # "repeat_penalty": 1.2,
    # Applies to the entire context, penalizes tokens that have already appeared in the context
    # by subtracting the presence_penalty value from their logit score if they have already appeared.
    # "presence_penalty": 1.5,
    # Frequency penalty follows the following formula, being less aggressive than presence penalty,
    # but still operating over the entire context:
    # new_logit_score = original_logit_score - (frequency_penalty * number_of_times_token_has_appeared)
    # "frequency_penalty": 1.0,
    # Prevent the model from generating newline characters
    # "penalize_newline": True,
    # Add custom stop sequences to halt generation
    # "stop": ["\n", "user:"],
    # Enable/disable numa for multi-socket systems
    # "numa": False,
    # The context window for the model
    # "num_ctx": 1024,
    # Reduces vram requirements for tokenization at the cost of speed with low values, inverse for higher values
    # "num_batch": 2,
    # Number of layers to offload to the GPU
    # "num_gpu": 1,
    # Define the main GPU if multiple are being used
    # "main_gpu": 0,
    # Most of the time should be kept True, uses memory-mapped files for model weights enabling
    # lazy loading of weights into VRAM
    # "use_mmap": True,
    # Set to number of CPU cores for faster inference (Time To First Token)
    "num_thread": 24
}

## Test Ollama Connection
Now let's test the model with the parameters and a simple question.

In [3]:
simple_question = "What is the capital of France?"

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": simple_question}
        ],
    options=parameters,
    stream=False
)

print(response.message.content)

The capital of France is **Paris**. It is the largest city in the country, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Seine River. Paris serves as the political, cultural, and economic center of France.


## Special Tokens
Next let's figure out the special tokens of the model. Make sure to change the `huggingface_model_name` to your model's name as it appears on the primary page for the model in the [Hub 🤗](https://huggingface.co). Ensure this is the normal safetensors version of the model and not the GGUF version. The code only downloads the tokenizer and not the full model file, and is only used in this step.

In [4]:
huggingface_model_name = "Qwen/Qwen3-1.7B"
# huggingface_model_name = "google/gemma-3-270m-it"

tokenizer = AutoTokenizer.from_pretrained(huggingface_model_name)
special_tokens = tokenizer.special_tokens_map
print(special_tokens)

{'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}


## Question Generation Sequence
We can use the following sequence of tokens to get the model to generate questions for us.

`BEGINNING OF SEQUENCE TOKEN` + `HEADER START` + "user" + `END OF HEADER`

This will cause the model to generate a question instead of a response like it typically would.

In [5]:
test_content = "The model should generate this!"

# just define the user for the message with no content
messages = [
    {"role": "user", "content": test_content}
]

# apply the chat template
prompt_sequence = tokenizer.apply_chat_template(
    messages,
    # We want to get the strings associated with the special tokens
    # not the ids so we set tokenize to false
    tokenize=False,
    # But we do want the generation prompt
    add_generation_prompt=True
)

print(prompt_sequence)

# split the prompt sequence into the beginning part we want
position_of_search = prompt_sequence.find(test_content)

question_generating_tokens = prompt_sequence[:position_of_search]

print(question_generating_tokens)

<|im_start|>user
The model should generate this!<|im_end|>
<|im_start|>assistant

<|im_start|>user



## Get a Question
Now let's try to get the model to generate a queston!

In [6]:
# Tell Ollama for this part we do not want the chat template to be applied
parameters["raw"] = True

# This is the default if not set by the user
previous_ctx = 1024

# save ctx if changed by the user
if "num_ctx" in parameters:
    previous_ctx = parameters["num_ctx"]

# We want low context for question generation, the model will continue to address the question.
parameters["num_ctx"] = 256

# Debugging if automatic formatting doesn't work for some reason
# print(f"Sequence: {question_generating_tokens}")

response = ollama.generate(
    model=model,
    prompt=question_generating_tokens,
    options=parameters,
    raw=True,
    stream=False,
)

# Extract question
question = response.response.split("\n")[0]

# slighlty different response structure for ollama.generate
print(question)

What is the value of $x$ in the equation $ \frac{1}{x} + \frac{1}{x} + \frac{1}{x} = \frac{1}{x} $?


## Get an Answer
We can now pass the question back to the model for the answer to it's question!

In [7]:
# we want to apply chat template again
parameters["raw"] = False

# restore context length
parameters["num_ctx"] = previous_ctx

# assume model supports thinking
model_thinks = True

try:
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "user", "content": question}
            ],
        options=parameters,
        # try thinking
        think=True,
        stream=False,
    )
except ollama.ResponseError as e:
    if "does not support thinking" in str(e):
        print("Model does not support thinking, retrying without it.")
        response = ollama.chat(
            model=model,
            messages=[
                {"role": "user", "content": question}
                ],
            options=parameters,
            stream=False,
        )

        model_thinks = False

print("Question:")
print(question)
print("-"*20)
print("Answer:")
print(response.message.content)

Question:
What is the value of $x$ in the equation $ \frac{1}{x} + \frac{1}{x} + \frac{1}{x} = \frac{1}{x} $?
--------------------
Answer:
To solve the equation:

$$
\frac{3}{x} = \frac{1}{x}
$$

we begin by eliminating the denominators. Since both sides have the same denominator $ x $, we can cross-multiply:

$$
3 \cdot x = 1 \cdot x
$$

Simplifying:

$$
3x = x
$$

Subtract $ x $ from both sides:

$$
3x - x = 0 \quad \Rightarrow \quad 2x = 0
$$

Divide both sides by 2:

$$
x = 0
$$

---

However, we must be cautious: the original equation contains $ \frac{1}{x} $, which is undefined when $ x = 0 $. Therefore, $ x = 0 $ is not a valid solution because it makes the expression undefined.

Now, consider the equation again:

$$
\frac{3}{x} = \frac{1}{x}
$$

We subtract $ \frac{1}{x} $ from both sides:

$$
\frac{3}{x} - \frac{1}{x} = 0 \quad \Rightarrow \quad \frac{2}{x} = 0
$$

This implies that $ \frac{2}{x} = 0 $, but a fraction can never equal zero unless the numerator is zero. Here, th

## Thinking
If desirable we can also extract the model's Chain of Thought as well.

In [8]:
if model_thinks:
    print(response.message.thinking)
else:
    print("Model does not support thinking.")

Okay, so I need to solve the equation $\frac{1}{x} + \frac{1}{x} + \frac{1}{x} = \frac{1}{x}$. Let me think about how to approach this. 

First, let me rewrite the equation to make it clearer. The left side has three terms of $\frac{1}{x}$ added together, and the right side is just $\frac{1}{x}$. So, combining the left side, it's like saying $3 \times \frac{1}{x} = \frac{1}{x}$. 

Hmm, so if I have three times something equals one times something else. Maybe I can subtract $\frac{1}{x}$ from both sides to get the equation in a simpler form. Let me try that:

Starting with:
$$
\frac{1}{x} + \frac{1}{x} + \frac{1}{x} = \frac{1}{x}
$$
Subtract $\frac{1}{x}$ from both sides:
$$
\frac{1}{x} + \frac{1}{x} + \frac{1}{x} - \frac{1}{x} = \frac{1}{x} - \frac{1}{x}
$$
Simplifying both sides:
Left side: $\frac{1}{x} + \frac{1}{x} + \frac{1}{x} - \frac{1}{x} = \frac{1}{x} + \frac{1}{x} = \frac{2}{x}$
Right side: $\frac{1}{x} - \frac{1}{x} = 0$

So now the equation is $\frac{2}{x} = 0$. 

Wait, but 

## Distill instruction/answer pairs
Now we can put it all together and distill a CSV with as much data as we like from our model! 

In [ ]:
dataset_size = 5
dataset_name = model + "_distilled_dataset.json"

for i in tqdm(range(dataset_size)):
    # Tell Ollama for this part we do not want the chat template to be applied
    parameters["raw"] = True

    parameters["num_ctx"] = 256

    # Generate question
    q_response = ollama.generate(
        model=model,
        prompt=question_generating_tokens,
        options=parameters,
        raw=True,
        stream=False,
    )

    question = q_response.response.split("\n")[0]

    parameters["num_ctx"] = previous_ctx

    # Generate answer
    a_response = ollama.chat(
        model=model,
        messages=[
            {"role": "user", "content": question}
            ],
        options=parameters,
        think=model_thinks,
        stream=False,
    )

    entry = {
        "instruction": question,
        "output": a_response.message.content
    }

    if model_thinks:
        entry["thinking"] = a_response.message.thinking
    
    with open(dataset_name, "a") as f:
        json.dump(entry, f, indent=4)

100%|██████████| 5/5 [04:46<00:00, 57.21s/it]
